In [53]:
using LowLevelFEM

gmsh.initialize();

In [54]:
#gmsh.open("plate_with_hole_tria_1st_size-def.geo")
#gmsh.open("plate_with_hole_tria_1st_size-5.geo")
#gmsh.open("plate_with_hole_tria_1st_size-1.geo")

#gmsh.open("plate_with_hole_tria_1st_size-graded.geo")

#gmsh.open("plate_with_hole_tria_2nd_size-def.geo")
#gmsh.open("plate_with_hole_tria_3rd_size-def.geo")
#gmsh.open("plate_with_hole_tria_5th_size-def.geo")

#gmsh.open("plate_with_hole_quad_1st_size-def.geo")

#gmsh.open("plate_with_hole_quad_1st_size-5.geo")
#gmsh.open("plate_with_hole_quad_2nd_size-5.geo")

#gmsh.open("plate_with_hole_quad_5st_size-1-graded.geo")

#gmsh.open("plate_with_hole_quad_1st_size-regular.geo")
#gmsh.open("plate_with_hole_quad_2nd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_5th_size-regular.geo")

#gmsh.open("plate_with_hole_quad_1st_size-regular.geo")
gmsh.open("plate_with_hole_quad_2nd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_3rd_size-regular.geo")
#gmsh.open("plate_with_hole_quad_5th_size-regular.geo")

#openPreProcessor()

mat = material("body")
problem = Problem([mat], type=:PlaneStress)

Info    : Reading 'plate_with_hole_quad_2nd_size-regular.geo'...
Info    : Meshing 1D...nts                                                                                                          
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.00168959s, CPU 0.00166s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 60%] Meshing surface 2 (Transfinite)
Info    : Done meshing 2D (Wall 0.000236095s, CPU 0.000235s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 20%] Meshing curve 2 order 2
Info    : [ 30%] Meshing curve 3 order 2
Info    : [ 40%] Meshing curve 4 order 2
Info    : [ 50%] Meshing curve 5 order 2
Info    : [ 60%] Meshin

Problem("plate_with_hole_quad_2nd_size-regular", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 525)

In [55]:
supp1 = displacementConstraint("bottom", uy=0)
supp2 = displacementConstraint("left", ux=0)
load1 = load("top", fy=1);

In [56]:
q = solveDisplacement(problem, [load1], [supp1, supp2])
S = solveStress(q);

In [57]:
#u = showDoFResults(q, name="uvec")
#ux = showDoFResults(q, :ux, name="ux")
#uy = showDoFResults(q, :uy, name="uy")

s1 = showStressResults(S, name="σ", visible=true, smooth=false)
s2 = showStressResults(S, name="σ smooth", smooth=true)
#sx = showStressResults(S, :sx, name="σx", smooth=true)
#sy = showStressResults(S, :sy, name="σy", smooth=true)
#synsm = showStressResults(S, :sy, name="σy non-smooth", smooth=false)
#sxy = showStressResults(S, :sxy, name="τxy", smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [58]:
gmsh.view.option.setNumber(s1, "NormalRaise", 10)
gmsh.view.option.setNumber(s2, "NormalRaise", 10)

gmsh.view.option.setNumber(s1, "ShowElement", 1)
gmsh.view.option.setNumber(s2, "ShowElement", 1)

In [59]:
err = fieldError(S)
serr = showDoFResults(err, name="szakadási hiba")

2

In [60]:
f3D = -S ⋅ ∇
f2D = projectTo2D(f3D)
showDoFResults(f2D, name="belső erők hibája")

3

In [61]:
n1y = VectorField(problem, "top", [0, 1, 0])
S1 = elementsToNodes(S)
S2 = nodesToElements(S1, onPhysicalGroup="top")
p3D = S2 * n1y

n1x = VectorField(problem, "top", [1, 0, 0])
p1x = p3D * n1x
p1y = p3D * n1y
f1f = load("top", fx=p1x, fy=p1y)
f2D = loadVector(problem, [f1f])
f = loadVector(problem, [load1])

showDoFResults(f, name="eredeti terhelés")
showDoFResults(f2D, name="terhelés feszültségből")
showDoFResults(f - f2D, name="din. peremfelt. hiba")

6

In [62]:
vx(x, y, z) = -x / √(x^2 + y^2)
vy(x, y, z) = -y / √(x^2 + y^2)
n2 = VectorField(problem, "hole", [vx, vy, 1])
S3 = nodesToElements(S1, onPhysicalGroup="hole")
p2 = S3 * n2
showElementResults(projectTo2D(p2), name="din. peremfelt. hiba - furat")

7

In [63]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [64]:
gmsh.finalize()

In [65]:
N = 1
σr = -N / 2 * (3 - 4 + 1)
σφ = N / 2 * (1 + 1) + N / 2 * (3 + 1)

3.0